In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
WorkflowMgm.Init("SolverPerformanceDrivenCavity");

In [1]:
using System.IO;
using BoSSS.Application.IBM_Solver;
using System.Data;

In [1]:
string dateString = "2018-5-17";
string File       = "SolverRuns--" + dateString + ".json";
string FilePath   = Path.Combine(CurrentDocDir, File);
var SessTab       = TableExtensions.LoadFromFile(FilePath);

In [1]:
static class Utils {
    // DOF per cell in 3D
    public static long Np(long p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }
}

In [1]:
// ====================
// Evaluation, plotting
// ====================

In [1]:
SessTab.GetColumnNames();

In [1]:
var SubTab_1 = SessTab.ExtractColumns("Grid:NoOfCells","SlvIter_excl","NonlinearSolve","LinearSolve","NoOfMultigridLevels","DGdegree:Velocity*","RegularTerminated");

In [1]:
// Add columns to DataTable
SubTab_1.Columns.Add("TotalDoFs",typeof(double));
long[] cells = ((long[]) SubTab_1.GetColumn("Grid:NoOfCells"));
long[] k = ((long[]) SubTab_1.GetColumn("DGdegree:Velocity*"));
int i     = 0;
foreach (DataRow row in SubTab_1.Rows) {
    row["TotalDoFs"] = (3*Utils.Np(k[i])+Utils.Np(k[i]-1))*cells[i];
    i++;
}

In [1]:
SubTab_1.Print();

In [1]:
static public class Formats {
static public void getFormat(int iTabRow,IDictionary<string, object> Row, 
                 out string Nmn, out PlotFormat Fmt){
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
         
           NonlinearSolverCodes Nonlin_solver_name = (NonlinearSolverCodes) Convert.ToInt32(Row["NonlinearSolve"]);
           LinearSolverCodes solver_name = (LinearSolverCodes) Convert.ToInt32(Row["LinearSolve"]);
           int MGLevels                  = Convert.ToInt32(Row["NoOfMultigridLevels"]);
           Fmt                           = new PlotFormat();
           
		   // ===!!!=== CAUTION ===!!!===
		   // Solverframework (SolverEnum numbers) have changed! Please check LinearSolver and NonLinearSolver for details
		   
           switch(Nonlin_solver_name) {
           case NonlinearSolverCodes.Picard: 
               Nmn = "Picard "; 
               break;  
           case NonlinearSolverCodes.Newton: 
               Nmn = "Newton "; 
               break; 
           case NonlinearSolverCodes.NewtonGMRES: 
               Nmn = "NewtonGmres "; 
               break;          
           default:
               Console.WriteLine("unknown: " + Nonlin_solver_name); 
               Nmn = "unknown"; 
               break;
           } 

          
           switch(solver_name) {
           case LinearSolverCodes.automatic: 
               Nmn           = Nmn+"Automatic"; 
               Fmt.PointType = PointTypes.OpenCircle;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCodes.direct_mumps: 
               Nmn           = Nmn+"Mumps"; 
               Fmt.PointType = PointTypes.OpenUpperTriangle;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCodes.direct_pardiso: 
               Nmn           = Nmn+"Pardiso"; 
               Fmt.PointType = PointTypes.Asterisk;
               Fmt.DashType  = DashTypes.Dashed;
               break;
           case LinearSolverCodes.exp_schwarz_directcoarse: 
               Nmn           = Nmn+"Swz w Coarse"; 
               Fmt.PointType = PointTypes.UpperTriangle;
               Fmt.DashType  = DashTypes.DotDashed;
               break;
           case LinearSolverCodes.exp_schwarz_Kcycle_directcoarse_overlap: 
               Nmn           = Nmn+"Swz Kcycle w Coarse Overlap"; 
               Fmt.PointType = PointTypes.Plus;
               Fmt.DashType  = DashTypes.Dashed;
               break;
           case LinearSolverCodes.exp_schwarz_directcoarse_overlap: 
               Nmn           = Nmn+"Swz w Coarse Overlap"; 
               Fmt.PointType = PointTypes.OpenBox;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCodes.exp_schwarz_Kcycle_directcoarse: 
               Nmn           = Nmn+"Swz Kcycle w Coarse"; 
               Fmt.PointType = PointTypes.LowerTriangle;
               Fmt.DashType  = DashTypes.DotDotDashed;
               break;
           case LinearSolverCodes.exp_softgmres: 
               Nmn           = Nmn+"SoftGMRES"; 
               Fmt.PointType = PointTypes.Box;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCodes.exp_softgmres_schwarz_directcoarse: 
               Nmn           = Nmn+"SoftGMRES Swz w Coarse"; 
               Fmt.PointType = PointTypes.OpenCircle;
               Fmt.DashType  = DashTypes.Dashed;
               break;
          case LinearSolverCodes.exp_softgmres_schwarz_directcoarse_overlap: 
               Nmn           = Nmn+"SoftGMRES Swz w Coarse Overlap"; 
               Fmt.PointType = PointTypes.OpenCircle;
               Fmt.DashType  = DashTypes.Dashed;
               break;
          case LinearSolverCodes.exp_softgmres_schwarz_Kcycle_directcoarse_overlap: 
               Nmn           = Nmn+"SoftGMRES Swz Kcycle w Coarse Overlap"; 
               Fmt.PointType = PointTypes.Times;
               Fmt.DashType  = DashTypes.DotDashed;
               break;
           default:
               Console.WriteLine("unknown: " + solver_name); 
               Nmn = Nmn+"unknown"; 
               break;
           } 
           Nmn           = Nmn;//+" MGLevels"+MGLevels;
           Fmt.PointSize = 0.5;
           Fmt.Style     = Styles.LinesPoints;
           Fmt.LineColor = LineColors.Black;
           Fmt.LineWidth = 3;
       }
}

In [1]:
var plotSlvIterTime = SubTab_1.ToPlot("TotalDoFs", "SlvIter_excl", // column for x- and y
                                  Formats.getFormat); // column for group
//var plotSlvIterSpeedup = SubTab_1.ToPlot("Grid:NoOfCells", "SlvIter_speedup", // column for x- and y
  //                                Formats.getFormat); // column for group
//var plotSlvInit = SubTab_1.ToPlot("Processors", "SlvInit_excl", // column for x- and y
//                                  Formats.getFormat); // column for group
Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
//plotSlvIterTime.Title     = "Exclusiveb_1.ToPlot("Grid:NoOfCells", "SlvIter_excl", // column for x- and ytimes";
plotSlvIterTime.Ylabel = "Time [s]";
plotSlvIterTime.Xlabel = "TotalDoFs";


plotSlvIterTime.XrangeMin = 1000;
plotSlvIterTime.XrangeMax = 1000000;
plotSlvIterTime.LogX      = true;
plotSlvIterTime.LogY      = true;
plotSlvIterTime.LogBaseX  = 10;
plotSlvIterTime.LogBaseY  = 10;

multiplots[0,0] = plotSlvIterTime;
//multiplots[1,0] = plotSlvIterSpeedup;

double[] dof = new[]{1e3,1e6};
double[] linT = dof.Select(x => x*1).ToArray();
var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
linP.Format.FromString("- black");
ArrayTools.AddToArray(linP, ref multiplots[0,0].dataGroups);

multiplots[0,0].bmargin = 4;
//multiplots[1,0].tmargin = 1;
multiplots[0,0].tmargin = 4;
multiplots.PlotNow();

In [1]:
LastError

In [1]:
var CL = multiplots.ToGnuplot().PlotCairolatex(xSize:18, ySize:10);

In [1]:
CL.SaveTo(Path.Combine(CurrentDocDir+"/plots/", "NodePerformance.tex"));

In [1]:
CL.PlotNow();

In [1]:
LastError